<a href="https://colab.research.google.com/github/fkonrad97/Network/blob/main/routingtable.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [37]:
import csv
import pandas as pd
import numpy as np
import networkx as nx
import random
from scipy.stats import entropy

# I/O

In [38]:
# Read positions of nodes (X,Y,Z)

def positionRead(name):
    positions = pd.read_csv(name + ".csv", header=None, sep=";")
    # Remove a plus sign from the end of the number
    positions[0][0] = positions[0][0][:-1]
    positions[0] = positions[0].astype(float)    # Convert data to numerical value
    return positions

In [39]:
# Read connection table between nodes

def connectionRead(name):
    connections = pd.read_csv(name + ".csv", header=None)
    return connections

In [40]:
# List of the positions of nodes
# positions = positionRead("Network/Brain_data/Brain1Positions")
positions = positionRead("/content/Brain1Positions")

# List of how nodes connected to each other
# connections = connectionRead("Network/Brain_data/Brain1Connections")
connections = connectionRead("/content/Brain1Connections")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


# Additional Methods

In [41]:
# Calculate distance between 2 points in Euclidean space
# Positions should be 3D (X,Y,Z)

def Euclidean_dist(p1, p2):
    point1 = np.array((positions[0][p1], positions[1][p1], positions[2][p1]))
    point2 = np.array((positions[0][p2], positions[1][p2], positions[2][p2]))
    return np.linalg.norm(point1 - point2)

In [42]:
# Add a key-value pair to the existing dictionary

def add_element(dict, key, value):
    if key not in dict:
        dict[key] = []
    dict[key].append(value)

In [43]:
# Returns with a dictionary. Key is the number of the node and 
# the value is the distant between the current node(act) and its neighbour.
# The list is ascendant by the distance.

def get_dist(act, neighbours):
    dist = {}
    
    for i in neighbours:
        add_element(dist, i, Euclidean_dist(act, i))
    
    dist = sorted(dist.items(), key=lambda x: x[1])
    
    return dist

In [44]:
def chooseRandomShortest(paths):
  randomPaths = []

  for i in paths:
    randomPaths.append(random.choice(i))

  return randomPaths

In [45]:
def avgEntropy(graph, routingTables):
  sum = 0

  for i in range(len(graph.nodes)):
    routingTables[i].getNextHopsList()
    sum += entropy(routingTables[i].nextHops)
  
  return (sum / len(graph.nodes))

In [46]:
def readWeightedIn():
  greedyNeighbours = []
  for i in range(len(connections)):
    for j in range(len(connections)):
       if connections[i][j] == 1 and i < j:
          dist = Euclidean_dist(i,j)
          greedyNeighbours.append([i,j,dist])
  return greedyNeighbours

In [47]:
def readUnweightedIn():
  neighbours = []
  for i in range(len(connections)):
    for j in range(len(connections)):
      if connections[i][j] == 1 and i < j:
        neighbours.append([i, j])
  return neighbours

# Graph algorithms

In [48]:
def findPaths(src, graph):
  paths = []

  for j in range(len(graph.nodes)):
    if src!=j:
      path = [p for p in nx.all_shortest_paths(graph, source=src, target=j)]
      paths.append(path)
  
  return paths

In [49]:
def greedy(graph, src, dest):
  pathsGreedy = []
  pathsGreedy.append(src)
  ready = False
  act = src

  while ready==False:
    nbours = [n for n in graph.neighbors(act)]

    # Calculates the distances of the neighbour nodes from the end node
    dist = get_dist(dest, nbours)

    # if the 'dest' is connected with 'act'
    if dest in nbours:
      pathsGreedy.append(dest)
      return pathsGreedy

    # if 'act' has no neighbours 
    elif len(nbours) == 0:
      return "No neighbours"

    # if 'act' has just 1 neighbour
    elif len(neighbours) == 1:
      act = nbours[0]
      pathsGreedy.append(act)

    elif len(nbours) > 1:
      not_found = False
      for i in dist:
        if i[0] not in pathsGreedy:
          act = i[0]
          pathsGreedy.append(act)
          not_found = True
          break
      # If there are no more choosable node on the list
      if not_found == False:
        act = dist[0][0]
        pathsGreedy.append(act)

# Classes

In [80]:
class Graph():
  def __init__(self,):
    self.G = nx.Graph()
    self.nodes = self.G.nodes

  def addWeightedEdges(self, neighborList):
    for i in neighborList:
      self.G.add_edge(i[0], i[1], weight=i[2])

  def addUnweightedEdges(self, neighborList):
    for i in neighborList:
      self.G.add_edge(i[0],i[1])

In [81]:
class routingTable():

  def __init__(self, nodeID, graph):
    self.graph = graph.G
    self.nodeID = nodeID
    self.table = []
    self.nextHops = []

  def addtoTable(self, dest, nextHop):
    self.table.append([dest, nextHop])

  def findNextHop(self):
    paths = findPaths(self.nodeID, self.graph)
    paths = chooseRandomShortest(paths)
    for i in paths:
      self.addtoTable(i[-1], i[1])

  def getNextHopsList(self):
    self.nextHops = []
    for i in self.table:
      self.nextHops.append(i[1])

# **GREEDY ALGORITHM**

In [73]:
Ggreedy = Graph()
greedyNeighbours = readWeightedIn()
Ggreedy.addWeightedEdges(greedyNeighbours)

In [74]:
pathsGreedy = []

for i in range(len(Ggreedy.nodes)):
  temp = []
  for j in range(len(Ggreedy.nodes)):
    if i!=j:
      temp.append(greedy(Ggreedy.G,i,j))
  pathsGreedy.append(temp)

In [64]:
routingGreedyTable = []

for i in pathsGreedy:
  temp = []
  for j in i:
    temp.append([j[-1], j[1]])
  routingGreedyTable.append(temp)

In [65]:
greedyNextHops = []

for i in routingGreedyTable:
  temp = []
  for j in i:
    temp.append(j[1])
  greedyNextHops.append(temp)

In [68]:
sum = 0

for i in range(len(Ggreedy.nodes)):
  sum += entropy(greedyNextHops[0])

avgGreedyEntropy = sum / len(Ggreedy.nodes)
avgGreedyEntropy

4.102882086518131

# **RANDOM UNWEIGHTED**

In [82]:
G = Graph()
neighbours = readUnweightedIn()
G.addUnweightedEdges(neighbours)

In [83]:
# Each nodes' routing table
routingTables = []

for j in range(10):
  routingtable = []
  for i in range(len(G.nodes)):
    table = routingTable(i,G)
    table.findNextHop()
    routingtable.append(table)
  routingTables.append(routingtable)

In [84]:
entropyList = []

for i in routingTables:
  entropyList.append(avgEntropy(G, i))

In [85]:
entropyList

[4.245982423362341,
 4.250469848455398,
 4.247939080793474,
 4.246330849763421,
 4.245614763704439,
 4.243350411067987,
 4.246620776826123,
 4.25031940012612,
 4.248992561418494,
 4.246167628965554]

# **KRIUKOV GRÁFGENERÁLÁS**